In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display
# !pip install -U pandasql
from pandasql import sqldf
mysql = lambda q: sqldf(q, globals())

warnings.filterwarnings("ignore")

In [15]:


# Extract the data

country_list = ["Bolivia", "Burundi", "Canada", "Chad", "Indonesia", "Mexico", "Niger", "Philippines", "USA"]
begin_date = 2005
end_date = 2020
nb_dates = end_date - begin_date + 1

attribute_names = ["Maternal_Leave_Benefits",
                    "Access_To_Basic_Drinking_Water_Total",
                    "Access_To_Basic_Drinking_Water_Urban",
                    "Access_To_Basic_Sanitation_Total",
                    "Access_To_Basic_Sanitation_Urban",
                    "Access_To_Managed_Drinking_Water_Total",
                    "Access_To_Managed_Sanitation_Total",
                    "Access_To_Basic_Handwashing_Facilities_Total",
                    "Access_To_Basic_Handwashing_Facilities_Urban",
                    "Unemployment_Female",
                    "Unemployment_Male",
                    "Unemployment_Total"
                    ]

nb_attributes = len(attribute_names)

QOL_data_raw = pd.read_csv("QOL_data_raw.csv")

QOL_transposed = QOL_data_raw.transpose()#flip dataframe

QOL_dimension = QOL_transposed.iloc[2,0:nb_attributes].transpose()#create data frame header columns
QOL_dimension = pd.DataFrame(QOL_dimension.values.reshape(1,-1))

count = 0
for k in range(len(country_list)):
    transposed_country = QOL_transposed.iloc[4+begin_date-1960:4+end_date-1960+1, count: count+nb_attributes]
    transposed_country.columns = range(transposed_country.columns.size)
    QOL_dimension = pd.concat([QOL_dimension, transposed_country], axis=0, ignore_index=True)
    count += nb_attributes


# add dates
date_table = pd.DataFrame(['Date'])
count = 0
for k in range(len(country_list)):
    date_table = pd.concat([date_table, pd.DataFrame([np.arange(begin_date, end_date+1)]).transpose()], axis=0)
date_table = date_table.reset_index().iloc[: , 1:]

QOL_dimension = pd.concat([date_table, QOL_dimension], axis=1, ignore_index=True)

QOL_dimension = QOL_dimension.drop(0)


country_table = pd.DataFrame(['Country'])
count = 0
for k in range(len(country_list)):
    country_table = pd.concat([country_table, pd.DataFrame([country_list[k]]*nb_dates)])
country_table = country_table.reset_index().iloc[: , 1:]

QOL_dimension = pd.concat([country_table, QOL_dimension], axis=1, ignore_index=True)

QOL_dimension = QOL_dimension.drop(0)


QOL_dimension.columns = ['Country_Ref', 'Date'] + attribute_names

QOL_dimension.insert(0, "QOL_Key",  np.arange(len(QOL_dimension)))

display(QOL_dimension.head(200))

# Save it into a new csv file
QOL_dimension.to_csv('QualityOfLife_dimension.csv', index = False, header=True)


,QOL_Key,Country_Ref,Date,Maternal_Leave_Benefits,Access_To_Basic_Drinking_Water_Total,Access_To_Basic_Drinking_Water_Urban,Access_To_Basic_Sanitation_Total,Access_To_Basic_Sanitation_Urban,Access_To_Managed_Drinking_Water_Total,Access_To_Managed_Sanitation_Total,Access_To_Basic_Handwashing_Facilities_Total,Access_To_Basic_Handwashing_Facilities_Urban,Unemployment_Female,Unemployment_Male,Unemployment_Total
1,0,Bolivia,2005,NaN,83.762849,96.318218,40.471657,52.04992,NaN,32.534432,NaN,NaN,3.103,2.115,2.544
2,1,Bolivia,2006,NaN,84.483153,96.505362,41.581381,52.912139,NaN,33.431655,NaN,NaN,3.208,2.085,2.576
3,2,Bolivia,2007,NaN,85.192934,96.692514,43.179455,54.36739,NaN,34.72126,NaN,NaN,3.371,2.102,2.646
4,3,Bolivia,2008,NaN,85.89325,96.879673,44.800626,55.841658,NaN,36.029204,NaN,NaN,3.386,2.001,2.6
5,4,Bolivia,2009,100.0,86.583198,97.066839,46.443766,57.334945,NaN,37.354599,NaN,NaN,3.622,2.468,2.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,139,USA,2016,NaN,99.537139,99.842653,99.747554,99.841607,96.59864,98.311762,NaN,NaN,4.789,4.939,4.87
141,140,USA,2017,NaN,99.625816,99.864026,99.729968,99.839976,96.782652,98.299286,NaN,NaN,4.312,4.402,4.36
142,141,USA,2018,NaN,99.713121,99.885399,99.712774,99.838346,96.965188,98.287095,NaN,NaN,3.841,3.951,3.9
143,142,USA,2019,NaN,99.799044,99.906772,99.695994,99.836715,97.146243,98.275223,NaN,NaN,3.611,3.721,3.67
